# 🧪 Clinical Lab SLA Analysis Report
**Author:** Allen Stalcup  
**Role:** Medical Laboratory Technician → Data Analyst Pivot  
**Tools:** PostgreSQL, Python (psycopg2, pandas, matplotlib), Jupyter  

---
### **Objective**
Analyze sample laboratory turnaround performance (TAT) and SLA compliance using SQL-based analytics.
The project simulates a reference lab environment where specimens arrive throughout the day,  
and performance is monitored by **shift**, **site**, and **analyte type**.

---
### **Key Questions**
1. How does SLA compliance vary by **shift** (Day, Evening, Night)?
2. Which **sites** consistently meet SLA targets?
3. What analytes show **high or low turnaround percentiles** relative to SLA?
4. How can visualization highlight throughput bottlenecks?

---
### **Tech Stack**
- **Database:** PostgreSQL (synthetic dataset under `synth` schema)
- **Queries:** Modular `.sql` files
- **Notebook:** Python Jupyter for visualization + storytelling
- **Visualization:** Matplotlib (no hard-coded colors to ensure portability)

## 🧩 Data Model Overview
The database includes 3 key tables:

| Table | Description |
|-------|--------------|
| `synth.specimens` | Tracks received and collected timestamps per sample |
| `synth.results` | Contains verified test results linked to specimens |
| `synth.analytes` | Defines SLA targets and test metadata |

Each result joins a specimen (via `specimen_id`) and an analyte (via `analyte_code`).


## Environment + installs

In [ ]:
# If running in a fresh environment, uncomment:
# %pip install psycopg2-binary pandas matplotlib python-dotenv
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import psycopg2
load_dotenv()  # loads .env if present

# Folders
OUT = Path("../artifacts"); OUT.mkdir(parents=True, exist_ok=True)
FIG = OUT / "figs"; FIG.mkdir(parents=True, exist_ok=True)
CSV = OUT / "csv";  CSV.mkdir(parents=True, exist_ok=True)


In [ ]:
%pip install python-dotenv

import os
from pathlib import Path
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
from dotenv import load_dotenv

# Load environment variables from .env file (kept local / private)
load_dotenv()

# Reusable database query function
def run_query(sql: str) -> pd.DataFrame:
    conn = psycopg2.connect(
        host=os.getenv("PG_HOST"),
        port=os.getenv("PG_PORT"),
        dbname=os.getenv("PG_DB"),
        user=os.getenv("PG_USER"),
        password=os.getenv("PG_PASSWORD")
    )
    df = pd.read_sql(sql, conn)
    conn.close()
    return df


### DB connection + helper

In [ ]:
from dotenv import load_dotenv
load_dotenv()


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="path/to/.env")


print("PG_HOST:", os.getenv("PG_HOST"))
print("PG_DB:", os.getenv("PG_DB"))
print("PG_USER:", os.getenv("PG_USER"))


In [ ]:
from dotenv import load_dotenv
import os
from pathlib import Path
import psycopg2
import pandas as pd

# --- Load environment variables ---
from dotenv import load_dotenv
load_dotenv()  # loads from current working directory


# --- Retrieve credentials ---
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DB = os.getenv("PG_DB")
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")

# --- Connection function ---
def run_sql(sql: str) -> pd.DataFrame:
    with psycopg2.connect(
        host=PG_HOST,
        port=PG_PORT,
        dbname=PG_DB,
        user=PG_USER,
        password=PG_PASSWORD
    ) as conn:
        return pd.read_sql(sql, conn)

print(f"Connected to {PG_DB} on {PG_HOST}:{PG_PORT} as {PG_USER}")



In [ ]:
# ... top of file (after imports)
from pathlib import Path

# existing OUT dir not required for images anymore; keep if you want CSVs there
VIS = Path("./visuals")
VIS.mkdir(parents=True, exist_ok=True)


---

### Canonical SLA views (quick pull)

**What:** Load standardized SLA metrics from SQL views so we slice consistently.
- `synth.sla_shift_v` → SLA by **shift** (Day/Evening/Night)
- `synth.sla_site_v`  → SLA by **site** (ClinicA/B/C)

**How SLA is calculated (once, in SQL):**

SLA % = 100 * AVG((verified_ts - received_ts <= tat_target_minutes)::int)
Also returns `avg_tat_min` and `n` (row count).

**Read the table:**
- Higher `sla_hit_pct` = better compliance
- Use `n` to judge stability (tiny n can be noisy)

**Quick take from this pull:** Night ≈ best, Evening ≈ solid, Day ≈ lowest (peak intake window).


In [ ]:
df_shift = run_sql("SELECT * FROM synth.sla_shift_v;")
df_site  = run_sql("SELECT * FROM synth.sla_site_v;")

# Optional stricter order-level metric: uncomment if you created it
# df_order_shift = run_sql("SELECT * FROM synth.sla_order_shift_v;")

df_shift, df_site.head()


---

### QC Impact & Rolling Intake (Operational Pulse)

**Goal:**  
Measure how QC (quality control) failures and specimen intake volume affect turnaround performance.

- `SQL_QC_IMPACT` → Compares **average TAT** for results near a QC **fail** vs. normal operation.  
  - `near_fail = True` → Bench had a QC fail within 60 minutes of result verification.
  - Highlights how instability impacts workflow speed.

- `SQL_ROLLING` → Tracks **hourly intake trends** with a **6-hour rolling total**.  
  - Useful for visualizing throughput surges (e.g., afternoon specimen spikes).

**Output:**  
- `df_qc` → Average TAT + sample counts grouped by QC condition.  
- `df_roll` → Hourly received counts and rolling workload totals.



In [ ]:
SQL_QC_IMPACT = """
WITH j AS (
  SELECT a.bench,
         EXTRACT(EPOCH FROM (r.verified_ts - s.received_ts))/60 AS tat_lab_min,
         EXISTS (
           SELECT 1
           FROM synth.qc_events q
           WHERE q.bench = a.bench
             AND q.severity = 'fail'
             AND q.event_ts BETWEEN r.verified_ts - INTERVAL '60 minutes' AND r.verified_ts
         ) AS near_fail
  FROM synth.results r
  JOIN synth.specimens s USING (specimen_id)
  JOIN synth.analytes  a USING (analyte_code)
)
SELECT near_fail, ROUND(AVG(tat_lab_min),1) AS avg_tat, COUNT(*) AS n
FROM j
GROUP BY near_fail
ORDER BY near_fail;
"""

SQL_ROLLING = """
WITH timeline AS (
  SELECT generate_series(
           date_trunc('hour', MIN(received_ts)),
           date_trunc('hour', MAX(received_ts)),
           interval '1 hour'
         ) AS hr
  FROM synth.specimens
),
counts AS (
  SELECT t.hr, COUNT(*) AS received_count
  FROM timeline t
  JOIN synth.specimens s
    ON s.received_ts >= t.hr AND s.received_ts < t.hr + interval '1 hour'
  GROUP BY t.hr
)
SELECT hr,
       received_count,
       SUM(received_count) OVER (ORDER BY hr ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) AS rolling_6hr_total
FROM counts
ORDER BY hr;
"""

df_qc   = run_sql(SQL_QC_IMPACT)
df_roll = run_sql(SQL_ROLLING)

df_qc, df_roll.head()



**Takeaway:**  
QC disruptions double lab TAT (≈34 → 73 min). Rolling intake quantifies the load curve driving these variances.

Save tidy CSVs (portfolio-friendly)

In [ ]:
df_shift.to_csv(CSV / "sla_by_shift.csv", index=False)
df_site.to_csv(CSV / "sla_by_site.csv", index=False)
df_qc.to_csv(CSV / "qc_fail_impact.csv", index=False)
df_roll.to_csv(CSV / "rolling_6hr.csv", index=False)
CSV


### SLA by Shift (Result-Level)

**Goal:**  
Visualize turnaround performance across operational shifts to identify timing-related efficiency gaps.

**Query:**  
`synth.sla_shift_v` — aggregates result-level SLA compliance by **Day / Evening / Night** shift.

**Output:** 

Each bar represents the percentage of results verified within their SLA target (≤ tat_target_minutes).

**Insight:**  
- **Night shift:** 100% compliance — optimal performance with minimal load.  
- **Evening shift:** Slight dip (≈99%) — consistent but less efficient than Night.  
- **Day shift:** Noticeable drop (≈92%) — indicates workload congestion or resource strain during peak hours.

**Takeaway:**  
Day operations face the heaviest volume and longest turnaround variance. Scheduling or staffing calibration could lift SLA compliance closer to 98–100%.


In [ ]:
plt.figure(figsize=(7,4))
plt.bar(df_shift["shift"], df_shift["sla_hit_pct"])
plt.title("SLA Compliance by Shift (Result-Level)")
plt.xlabel("Shift")
plt.ylabel("SLA Compliance (%)")
plt.ylim(0, 100)

# Add value labels
for i, v in enumerate(df_shift["sla_hit_pct"]):
    plt.text(i, v + 1, f"{v:.1f}%", ha="center", va="bottom", fontsize=9)

plt.tight_layout()

# Show in notebook

# ... your plotting code (plt.bar, labels, etc.)
plt.tight_layout()

# Save FIRST, then show
plt.savefig(VIS / "fig_sla_by_shift.png", dpi=300,
            bbox_inches="tight", facecolor="white")
plt.show()
plt.close()









### Chart: SLA by site

In [ ]:
plt.figure(figsize=(8,4))
plt.barh(df_site["source_site"], df_site["sla_hit_pct"])
plt.title("SLA Compliance by Site (Result-Level)")
plt.xlabel("SLA Compliance (%)")
plt.ylabel("Source Site")

# Add value labels
for i, v in enumerate(df_site["sla_hit_pct"]):
    plt.text(v + 1, i, f"{v:.1f}%", va="center", fontsize=9)

plt.xlim(0, 100)
plt.tight_layout()


# ... your plotting code (plt.bar, labels, etc.)
plt.tight_layout()

# Save FIRST, then show
plt.savefig(VIS / "fig_sla_by_site.png", dpi=300,
            bbox_inches="tight", facecolor="white")
plt.show()
plt.close()





**Takeaway:**  
The ED likely faces heavier specimen volume or more complex workflows during peak hours.  
Prioritizing **resource balancing, staffing, or process automation** in the ED could restore overall SLA compliance to near-perfect levels.

Investigating wether the ED Lab intakes specimens from outside sources required. (Check for Cross-Site Transfers)

*If the collection-to-receipt delay > 60 minutes, it’s probably not collected on-site — i.e., transported from an external facility.*

Chart: QC fail impact

In [ ]:
labels = ["Normal", "QC Fail"]
vals = []
for flag in [False, True]:
    row = df_qc[df_qc["near_fail"] == flag]
    vals.append(float(row["avg_tat"].iloc[0]) if not row.empty else 0.0)

plt.figure(figsize=(6,4))
plt.bar(labels, vals)
plt.title("QC Fail Proximity Impact on Lab TAT")
plt.ylabel("Average Lab TAT (minutes)")

# Add value labels
for i, v in enumerate(vals):
    plt.text(i, v + 1, f"{v:.1f}", ha="center", va="bottom", fontsize=9)

plt.tight_layout()


# Save FIRST, then show
plt.savefig(VIS / "fig_qc_fail_impact.png", dpi=300,
            bbox_inches="tight", facecolor="white")
plt.show()
plt.close()


**Takeaway:**  
QC incidents introduce measurable slowdowns — likely due to instrument recalibration, result verification delays, or reruns.  
Reducing QC event frequency or improving post-failure recovery could yield **>50% faster throughput** during critical periods.

Chart: Rolling 6-hour intake

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df_roll["hr"], df_roll["rolling_6hr_total"], linewidth=2)
plt.title("Rolling 6-Hour Specimen Intake Volume")
plt.xlabel("Hour")
plt.ylabel("Specimens Received (6hr rolling)")

# Format timestamps on X-axis neatly
plt.gcf().autofmt_xdate()

plt.tight_layout()


# Save FIRST, then show
plt.savefig(VIS / "fig_rolling_6hr.png", dpi=300,
            bbox_inches="tight", facecolor="white")
plt.show()
plt.close()



**Takeaway:**  
Operationally, this confirms a **stable daily rhythm** but highlights opportunities to **redistribute staffing or automation** during intake spikes to maintain SLA compliance.

---

**Goal** — Percentile-by-Analyte TAT vs. SLA

Compute P50/P90/P95 turnaround times per analyte (verified_ts − received_ts in minutes) and compare to each analyte’s SLA target. This reveals median performance, tail risk (P90/P95), and where tests are most likely to breach SLA, so we can prioritize workflow or capacity fixes.

In [ ]:
SQL_PCT = """
WITH m AS (
  SELECT a.analyte_code,
         EXTRACT(EPOCH FROM (r.verified_ts - s.received_ts))/60 AS tat_min,
         a.tat_target_minutes AS sla_min
  FROM synth.results r
  JOIN synth.specimens s USING (specimen_id)
  JOIN synth.analytes  a USING (analyte_code)
)
SELECT analyte_code,
       ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY tat_min)::numeric,1) AS p50,
       ROUND(PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY tat_min)::numeric,1) AS p90,
       ROUND(PERCENTILE_CONT(0.95)WITHIN GROUP (ORDER BY tat_min)::numeric,1) AS p95,
       MAX(sla_min) AS sla_min
FROM m
GROUP BY analyte_code
ORDER BY analyte_code;
"""
df_pct = run_sql(SQL_PCT)
df_pct.to_csv(CSV / "percentiles_by_analyte.csv", index=False)
df_pct

plt.figure(figsize=(8,5))

# Plot bars for percentiles
plt.bar(df_pct["analyte_code"], df_pct["p95"], alpha=0.6, label="P95")
plt.bar(df_pct["analyte_code"], df_pct["p90"], alpha=0.6, label="P90")
plt.bar(df_pct["analyte_code"], df_pct["p50"], alpha=0.6, label="P50")

# Overlay SLA target line
plt.plot(df_pct["analyte_code"], df_pct["sla_min"], color="red", linewidth=2, label="SLA Target")

plt.title("Turnaround Time Percentiles by Analyte")
plt.xlabel("Analyte")
plt.ylabel("Minutes")
plt.legend()


# ... your plotting code (plt.bar, labels, etc.)
plt.tight_layout()

# Save FIRST, then show
plt.savefig(VIS / "tat_percentiles_by_analyte.png", dpi=300,
            bbox_inches="tight", facecolor="white")
plt.show()
plt.close()



Takeaway:
Turnaround is generally strong, but CMP and LIPID panels could benefit from workflow refinement or instrument load balancing to tighten their upper-percentile completion times.

A1C times likely affected by CBCs running before A1Cs on the same specimen.  

---

**SLA by Analyte × Shift**

**Goal**
See which tests (analytes) slip against SLA by time of day so you can target staffing, batching, or instrument scheduling.

**What the table shows**
Each row is an analyte; each shift (Day/Evening/Night) has:

avg_tat_min – average lab turnaround in minutes (receipt → verification)

sla_hit_pct – % of results meeting that analyte’s SLA

n – result count (sample size / confidence)

In [ ]:
# Step 3 — Load and execute the SQL query
sql_path = Path.cwd() / "sql" / "sla_by_analyte_shift.sql"

# Safety check to prevent empty query errors
SQL_SLA_ANALYTE_SHIFT = sql_path.read_text().strip()
if not SQL_SLA_ANALYTE_SHIFT:
    raise ValueError(f"{sql_path} is empty — recheck your SQL script")

# Run the query
df_heat = run_query(SQL_SLA_ANALYTE_SHIFT)
print("Rows returned:", len(df_heat))
df_heat.head()

# Pivot to a matrix: analyte (rows) × shift (cols) with SLA %
heat = (
    df_heat
      .pivot(index="analyte_code", columns="shift", values="sla_hit_pct")
      .reindex(columns=["Day", "Evening", "Night"])                # consistent column order
)

# Optional: sort rows by average SLA (best → worst)
heat = heat.loc[heat.mean(axis=1).sort_values(ascending=False).index]

# Make sure visuals folder exists
from pathlib import Path
VIS = Path("visuals"); VIS.mkdir(exist_ok=True)

# Plot heatmap
import matplotlib.pyplot as plt
plt.figure(figsize=(7, 4.5))
im = plt.imshow(heat.values, aspect="auto", vmin=85, vmax=100)      # bounded to highlight 85–100%
plt.colorbar(im, label="SLA compliance (%)")

# Axes ticks/labels
plt.xticks(range(heat.shape[1]), heat.columns)
plt.yticks(range(heat.shape[0]), heat.index)
plt.title("SLA Compliance by Analyte × Shift")

# Value annotations
for i in range(heat.shape[0]):
    for j in range(heat.shape[1]):
        val = heat.iloc[i, j]
        if pd.notna(val):
            plt.text(j, i, f"{val:.1f}%", ha="center", va="center", fontsize=9)

# ... your plotting code (plt.bar, labels, etc.)
plt.tight_layout()

# Save FIRST, then show
plt.savefig(VIS / "fig_sla_heatmap_analyte_shift.png", dpi=300,
            bbox_inches="tight", facecolor="white")
plt.show()
plt.close()


**Takeaway**
Day shift is the bottleneck. CBC (81.2%) and PT/INR (88.8%) miss the 95–97% target; CMP (97.4%), A1C (96.3%), and LIPID (95.0%) are borderline but okay.

---

In [ ]:
# ---------------------------
# Automated Textual Insights
# ---------------------------

def generate_sla_insights(df):
    insights = []
    overall_mean = df["sla_hit_pct"].mean()

    # Identify top and bottom performers
    top = df.sort_values("sla_hit_pct", ascending=False).head(3)
    low = df.sort_values("sla_hit_pct").head(3)

    insights.append(f"✅ Overall average SLA compliance across analytes and shifts: **{overall_mean:.2f}%**.\n")

    insights.append("🏆 **Top-performing combinations:**")
    for _, row in top.iterrows():
        insights.append(f"- {row['analyte_code']} ({row['shift']}): {row['sla_hit_pct']:.2f}% SLA compliance.")

    insights.append("\n⚠️ **Lowest-performing combinations:**")
    for _, row in low.iterrows():
        diff = overall_mean - row['sla_hit_pct']
        insights.append(f"- {row['analyte_code']} ({row['shift']}): {row['sla_hit_pct']:.2f}% "
                        f"(−{diff:.1f}% below mean).")

    # Shift-level summary
    shift_summary = (
        df.groupby("shift")["sla_hit_pct"]
        .mean()
        .sort_values(ascending=False)
        .reset_index()
    )

    insights.append("\n🕒 **Shift-level summary:**")
    for _, row in shift_summary.iterrows():
        insights.append(f"- {row['shift']}: {row['sla_hit_pct']:.2f}% average compliance.")

    return "\n".join(insights)


# Generate insights
text_report = generate_sla_insights(df_heat)
print(text_report)
